# [백준/모든 점을 포함하는 원](https://www.acmicpc.net/problem/13708)


## 풀이과정


### 첫번째 시도


#### 풀이과정


In [ ]:
from math import hypot
from typing import Sequence, Generator, Callable


class Point:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

    def __iter__(self) -> Generator[float, None, None]:
        yield from (self.x, self.y)

    def __sub__(self, other: "Point"):
        return Point(self.x - other.x, self.y - other.y)

    def __add__(self, other: "Point"):
        return Point(self.x + other.x, self.y + other.y)

    def __mul__(self, other: "Point"):
        # dot
        return self.x * other.x + self.y * other.y

    def __rmul__(self, other: float):
        # scalar
        return Point(self.x * other, self.y * other)

    def __truediv__(self, other: float):
        return Point(self.x / other, self.y / other)

    def __matmul__(self, other: "Point"):
        # cross
        return self.x * other.y - self.y * other.x

    def __rmatmul__(self, other: tuple["Point", "Point"]):
        # matrix multiplication
        return Point(self * other[0], self * other[1])

    def rev(self, other: "Point"):
        return (Point(other.y, -self.y), Point(-other.x, self.x))

    def __abs__(self):
        return hypot(self.x, self.y)


class Circle:
    def __init__(self, center: Point, radius: float, includes: set[Point] = set()):
        self.center = center
        self.radius = radius
        self.includes = includes

    def __repr__(self) -> str:
        return f"Circle({self.center}, {self.radius})"

    def __contains__(self, point: Point | Sequence[Point]) -> bool:
        if isinstance(point, Point):
            return abs(point - self.center) <= self.radius
        return all(p in self for p in point)

    @staticmethod
    def append_includes(func: Callable[["Circle", Point], "Circle"]):
        def inner(self: "Circle", point: Point):
            circle = func(self, point)
            circle.includes = self.includes | {point}
            return circle

        return inner

    @append_includes
    def __add__(self, point: Point) -> "Circle":
        if point in self:
            return self
        circle = Circle.from_2points(point, prev := next(points := iter(self.includes)))
        for p in points:
            if p not in circle:
                circle, prev = Circle.from_3points(point, prev, p), p
        return circle

    @staticmethod
    def from_2points(a: Point, b: Point) -> "Circle":
        return Circle((a + b) / 2, abs(a - b) / 2, {a, b})

    @staticmethod
    def get_center(a: Point, b: Point) -> Point:
        return a.rev(b) @ Point(a * a, b * b) / (2 * (a @ b))

    @staticmethod
    def from_3points(a: Point, b: Point, c: Point) -> "Circle":
        center = Circle.get_center(a - c, b - c)
        return Circle(center + c, abs(center))

    def __repr__(self):
        return f"{round(self.center.x, 3)} {round(self.center.y, 3)} {round(self.radius, 3)}"


def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = (Point(*map(float, i.split())) for i in inputs)
    circle: Circle = sum(points, start=Circle.from_2points(next(points), next(points)))
    print(round(circle.radius * 2, 2))


solution()

### 두번째 시도


#### 풀이과정

사실상 [2389번 문제](https://www.acmicpc.net/problem/2389)와 동일한 문제이기 때문에 [동일한 방식](https://www.acmicpc.net/source/71761185)으로 풀었다.


In [ ]:
# refer https://www.acmicpc.net/source/55045393


def cross(a: complex, b: complex) -> float:
    return (a.conjugate() * b).imag


def get_center_2(a: complex, b: complex) -> complex:
    return (a + b) / 2


def get_center_3(a: complex, b: complex, c: complex) -> complex:
    ab = b - a
    ac = c - a
    return a + (abs(ac) ** 2 * ab - abs(ab) ** 2 * ac) * 1j / (2 * cross(ab, ac))


def get_circle(bound: list[complex]):
    center = (get_center_3 if len(bound) > 2 else get_center_2)(*bound)
    return center, abs(center - bound[0])


def init(bound: list[complex]) -> tuple[complex, float]:
    return get_circle(bound) if len(bound) > 1 else ((bound[0] if bound else 0j), 0.0)


def enclosed(bound: list[complex], points: list[complex]) -> tuple[complex, float]:
    if len(bound) == 3:
        return get_circle(bound)
    center, radius = init(bound)
    for i, p in enumerate(points):
        if abs(center - p) > radius:
            center, radius = enclosed(bound + [p], points[:i])
    return center, radius


def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = list(set(complex(*map(float, line.split())) for line in inputs))
    _, radius = enclosed([], points)
    print("%.2f" % radius * 2)


solution()

![2024년 1월 15일 14시 기준 13708번 맞힌 사람 (Python) 40ms로 1위](<../../img/Screenshot 2024-01-15 at 20-03-16 13708번 맞힌 사람 (Python) - 1 페이지.png>)


## 해답


In [ ]:
def cross(a: complex, b: complex) -> float:
    return (a.conjugate() * b).imag

In [ ]:
def get_center_2(a: complex, b: complex) -> complex:
    return (a + b) / 2

In [ ]:
def get_center_3(a: complex, b: complex, c: complex) -> complex:
    ab = b - a
    ac = c - a
    return a + (abs(ac) ** 2 * ab - abs(ab) ** 2 * ac) * 1j / (2 * cross(ab, ac))

In [ ]:
def get_circle(bound: list[complex]):
    center = (get_center_3 if len(bound) > 2 else get_center_2)(*bound)
    return center, abs(center - bound[0])

In [ ]:
def init(bound: list[complex]) -> tuple[complex, float]:
    return get_circle(bound) if len(bound) > 1 else ((bound[0] if bound else 0j), 0.0)

In [ ]:
def enclosed(bound: list[complex], points: list[complex]) -> tuple[complex, float]:
    if len(bound) == 3:
        return get_circle(bound)
    center, radius = init(bound)
    for i, p in enumerate(points):
        if abs(center - p) > radius:
            center, radius = enclosed(bound + [p], points[:i])
    return center, radius

In [ ]:
def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = list(set(complex(*map(float, line.split())) for line in inputs))
    _, radius = enclosed([], points)
    print("%.2f" % radius * 2)

## 예제


In [264]:
# 백준 문제 풀이용 예제 실행 코드
from bwj import test

test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

In [265]:
test_solution(
    """4
1 1
1 0
0 1
0 0
"""
)  # 1.41

1.41
